# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, user-minimum-scale=1, maximum-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="501887343352051" property="fb:app_id"/>
<meta content="Yahoo奇摩電影" property="og:site_name"/>
<title>Yahoo奇摩電影</title>
<meta content="Yahoo奇摩電影" name="twitter:title"/>
<meta content="Yahoo奇摩電影" property="og:title"/>
<link href="https://movies.yahoo.com.tw" rel="canonical"/>
<meta content="https://movies.yahoo.com.tw" property="og:url"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="twitter:description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" name="description"/>
<meta content="提供最新的電影資訊、預告片、劇照、電影時刻表、明星資訊、網友期待榜、網友滿意榜、票房榜和電影相關動態" property="og:description"/>
<meta content="https://movies.yahoo.com.tw/build/images/ogImage.png" name="twitter:image"/>
<meta content="https://movies.yahoo

### 先搜尋全部的電影代號(ID)資訊

In [3]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})
for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 特約經紀公司, ID: 10364
Movie: 一屍到底番外篇：好萊塢大作戰！, ID: 10352
Movie: 餓魔餐廳, ID: 10342
Movie: 花生醬獵鷹的願望, ID: 10340
Movie: 女孩要革命, ID: 10339
Movie: 我的靈魂是愛做的, ID: 10336
Movie: 絕世情歌, ID: 10328
Movie: 人間失格：太宰治與他的3個女人, ID: 10287
Movie: 抓狂博物館, ID: 10279
Movie: 呆萌特務, ID: 10264
Movie: 去年聖誕節, ID: 10256
Movie: 野蠻遊戲：全面晉級, ID: 10134
Movie: 布紐爾超現實人生, ID: 10334
Movie: 夕霧花園, ID: 10327
Movie: 寂寞診療室, ID: 10282
Movie: 懸案密碼前傳：瓶中信, ID: 10254
Movie: 布魯克林孤兒, ID: 10224
Movie: 暴走曼哈頓, ID: 10044
Movie: 鋒迴路轉, ID: 10258
Movie: 賽道狂人, ID: 10097
Movie: 麂皮：永不滿足, ID: 10358
Movie: 82年生的金智英, ID: 10298
Movie: 許怨房, ID: 10294
Movie: BanG Dream！ FILM LIVE, ID: 10271
Movie: 冰雪奇緣2, ID: 9597
Movie: 催眠．裁決, ID: 10322
Movie: 抱歉我們錯過你了, ID: 10309
Movie: Hello World, ID: 10302
Movie: 大餓, ID: 10296
Movie: 萬萬沒想到, ID: 10281
Movie: 神機有毛病, ID: 10260
Movie: 為美好的世界獻上祝福！紅傳說, ID: 10226
Movie: 樂園, ID: 10280
Movie: 決戰中途島, ID: 10130
Movie: 安眠醫生, ID: 10098
Movie: 地久天長, ID: 9621
Movie: 魔鬼終結者：黑暗宿命, ID: 10067
Movie: 紫羅蘭永恆花園外傳－永遠與自動手記人偶－, ID: 10292
Movie: 

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [4]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10364

In [5]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

[{'area_id': 28, 'title': '台北市', 'priority': 150, 'ts_update': 1547519309}, {'area_id': 8, 'title': '新北市', 'priority': 145, 'ts_update': 1547194527}, {'area_id': 16, 'title': '桃園', 'priority': 140, 'ts_update': 1547452573}, {'area_id': 20, 'title': '新竹', 'priority': 135, 'ts_update': 1547452603}, {'area_id': 15, 'title': '苗栗', 'priority': 130, 'ts_update': 1547452620}, {'area_id': 2, 'title': '台中', 'priority': 125, 'ts_update': 1547507368}, {'area_id': 21, 'title': '嘉義', 'priority': 105, 'ts_update': 1547507403}, {'area_id': 10, 'title': '台南', 'priority': 100, 'ts_update': 1547507403}, {'area_id': 17, 'title': '高雄', 'priority': 95, 'ts_update': 1547507403}, {'area_id': 24, 'title': '金門', 'priority': 70, 'ts_update': 1547507437}, {'area_id': 23, 'title': '澎湖', 'priority': 65, 'ts_update': 1547507437}]
放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 嘉義, 代號(area_id): 21
放映地區: 

### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id = 2

In [7]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})
# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 12
十二月 13
十二月 14
十二月 15
十二月 16


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [8]:
# 選定要觀看的日期
date = "2019-12-12"

In [9]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [61]:
'''
  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
'''
data = soup.find_all(class_="area_time _c jq_area_time",attrs={'data-theater_name'})

for i in data:
    print('電影院: ' + i.li.a.string)
    print('影片放映類型:'+ i.find(class_="tapR").string)
    print("時間表: \n")
    for d in i.find_all(class_="gabtn"):
        print(d["value"])
    print("----------------------------------------------------------------------------------------\n\n")
'''
電影院
影片放映類型

'''

電影院: 日新大戲院
影片放映類型:數位
時間表: 

2019-12-12 10:00:00
2019-12-12 17:00:00
----------------------------------------------------------------------------------------


電影院: 台中大遠百威秀影城
影片放映類型:數位
時間表: 

2019-12-12 19:15:00
2019-12-12 22:55:00
----------------------------------------------------------------------------------------


電影院: 台中華威台中影城
影片放映類型:數位
時間表: 

2019-12-12 19:00:00
2019-12-12 23:30:00
----------------------------------------------------------------------------------------


電影院: 台中站前秀泰影城
影片放映類型:數位
時間表: 

2019-12-12 10:40:00
2019-12-12 12:50:00
2019-12-12 15:00:00
2019-12-12 17:10:00
2019-12-12 19:20:00
2019-12-12 21:30:00
2019-12-12 23:40:00
----------------------------------------------------------------------------------------


電影院: 台中文心秀泰影城
影片放映類型:數位
時間表: 

2019-12-12 11:30:00
2019-12-12 13:40:00
2019-12-12 15:50:00
2019-12-12 18:00:00
2019-12-12 20:10:00
2019-12-12 22:20:00
----------------------------------------------------------------------------------------


電影院: 豐原in89豪

'\n電影院\n影片放映類型\n\n'